## Preprocessing Comments

In [35]:
import os
import glob
import numpy as np
import pandas as pd
import re
from datetime import datetime
import pickle

In [15]:
path = '../data'

comment_folder = glob.glob(os.path.join(path, "reddit_swiss_comments_*"))
comments_files = [glob.glob(os.path.join(folder, "part-*")) for folder in comment_folder]
    
for indx in range(len(comments_files)):
    dfs_comments = pd.concat((pd.read_csv(f) for f in comments_files[indx]))
    dfs_comments.head()

In [23]:
dfs_comments.shape

(18349, 33)

In [25]:
dfs_comments.columns

Index(['approved_by', 'archived', 'author', 'author_flair_css_class',
       'author_flair_text', 'banned_by', 'body', 'body_html',
       'controversiality', 'created', 'created_utc', 'distinguished', 'downs',
       'edited', 'gilded', 'id', 'likes', 'link_id', 'mod_reports', 'name',
       'num_reports', 'parent_id', 'removal_reason', 'replies',
       'report_reasons', 'retrieved_on', 'saved', 'score', 'score_hidden',
       'subreddit', 'subreddit_id', 'ups', 'user_reports'],
      dtype='object')

In [28]:
dates = [datetime.fromtimestamp(t) for t in dfs_comments['created_utc'].astype(int)]
dfs_comments['dates'] = dates

In [29]:
deleted_comments = dfs_comments[dfs_comments['body'] != '[deleted]']

dfs_comments_clean = dfs_comments[dfs_comments['body'] != '[deleted]'].copy()
dfs_comments_clean.head()

,approved_by,archived,author,author_flair_css_class,author_flair_text,banned_by,body,body_html,controversiality,created,...,report_reasons,retrieved_on,saved,score,score_hidden,subreddit,subreddit_id,ups,user_reports,dates
0,null,False,renner2,null,null,null,"Its late, but was recently posted to the SBB b...",null,0,null,...,null,1425123847,null,1,False,Switzerland,t5_2qhp6,1,null,2015-01-01 01:35:02
1,null,False,SwissBliss,vd,Lived my whole life in Vaud/Family in Valais,null,Haha will do,null,0,null,...,null,1425123317,null,3,False,Switzerland,t5_2qhp6,3,null,2015-01-01 02:20:02
2,null,False,KikiManjaro,null,null,null,Happy new year,null,0,null,...,null,1425122850,null,3,False,Switzerland,t5_2qhp6,3,null,2015-01-01 03:01:13
3,null,False,[deleted],null,null,null,Better late than never. Thank you and happy ne...,null,0,null,...,null,1425120179,null,1,False,Switzerland,t5_2qhp6,1,null,2015-01-01 07:42:56
4,null,False,Kyrixia,null,null,null,"Im in verbier atm too, would love to ski :) pm...",null,0,null,...,null,1425120021,null,2,False,Switzerland,t5_2qhp6,2,null,2015-01-01 08:03:32


In [30]:
dfs_comments_clean.describe()

,controversiality,created_utc,gilded,score,ups
count,17660.000000,1.766000e+04,17660.000000,17660.000000,17660.000000
mean,0.013364,1.431279e+09,0.000566,2.917214,2.917214
std,0.114829,5.956188e+06,0.023790,4.359535,4.359535
min,0.000000,1.420073e+09,0.000000,-54.000000,-54.000000
25%,0.000000,1.426272e+09,0.000000,1.000000,1.000000
50%,0.000000,1.431824e+09,0.000000,2.000000,2.000000
75%,0.000000,1.436307e+09,0.000000,3.000000,3.000000
max,1.000000,1.441064e+09,1.000000,91.000000,91.000000


In [31]:
def findURL(line):
    pattern = r'(https?:\/\/(?:www\.|(?!www))[^\s\.]+\.[^\s]{2,}|www\.[^\s]+\.[^\s]{2,}|([-a-zA-Z0-9@:%._\+~#=]{2,256}\.[a-z]{2,6}\b([-a-zA-Z0-9@:%_\+.~#?&//=]*)))'
    match = re.search( pattern, line, re.I)
    if match:
        return match.group()
    else:
        return 'NULL'

In [32]:
urls = dfs_comments_clean.apply(lambda x: findURL(x['body']),axis=1)
dfs_comments_clean.loc[:,'extractedURL'] = urls
dfs_comments_clean.loc[:,'body'] = dfs_comments_clean.apply(lambda x: x['body'].replace(x['extractedURL'],''),axis=1)

In [33]:
dfs_comments_clean[dfs_comments_clean['extractedURL'] != 'NULL'].shape

(2251, 35)

In [34]:
comments = dfs_comments_clean['body'].values
comments

array(['Its late, but was recently posted to the SBB blog.\n\n',
       'Haha will do', 'Happy new year ', ...,
       "I didn't mean to and I'm sorry if it came across as such. I would never insinuate something like that.",
       "Don't forget Spiez!",
       'Yes, everyone has different personality and needs to adjust accordingly'], dtype=object)

In [36]:
pickle.dump(copen('comments_processed.pickle','wb'))

TypeError: Required argument 'file' (pos 2) not found